In [2]:
import pandas as pd
import numpy as np
import random

# Configuration
NUM_SAMPLES = 10000
BENIGN_RATIO = 0.5
OUTPUT_FILE = "fake_banking_app_dataset.csv"

# --- Feature Definitions ---
# 1. Permissions (Binary: 0 or 1)
PERMISSIONS = [
    'android.permission.INTERNET',
    'android.permission.READ_SMS',
    'android.permission.RECEIVE_SMS',
    'android.permission.READ_CONTACTS',
    'android.permission.SYSTEM_ALERT_WINDOW', # Critical for overlays
    'android.permission.REQUEST_INSTALL_PACKAGES', # Dropper behavior
    'android.permission.ACCESS_FINE_LOCATION',
    'android.permission.CAMERA',
    'android.permission.RECORD_AUDIO',
    'android.permission.GET_ACCOUNTS'
]

# 2. API Calls (Normalized Frequency: 0.0 to 1.0)
API_CALLS = [
    'api_telephony_getDeviceId',
    'api_sms_sendTextMessage',
    'api_crypto_Cipher',
    'api_reflection_Class_forName', # Obfuscation indicator
    'api_net_HttpURLConnection'
]

# 3. Metadata (Numeric/Binary)
METADATA = [
    'app_size_mb',       # Float
    'min_sdk',           # Int
    'target_sdk',        # Int
    'cert_age_days',     # Int (Newer certs often riskier)
    'is_debuggable',     # Binary
    'obfuscation_score'  # Float 0-1
]

# 4. Behavioral Signals (Simulated Scores: 0.0 to 1.0)
BEHAVIOR = [
    'background_service_score',
    'network_call_freq',
    'screen_overlay_score', # High value = Phishing risk
    'sms_trigger_score'     # High value = OTP theft risk
]

def generate_app_profile(is_malicious):
    profile = {}

    # --- Label ---
    profile['label'] = 1 if is_malicious else 0

    # --- Permissions Generation ---
    for perm in PERMISSIONS:
        if is_malicious:
            # Malicious apps have higher probability of dangerous perms
            if perm in ['android.permission.READ_SMS', 'android.permission.RECEIVE_SMS', 'android.permission.SYSTEM_ALERT_WINDOW']:
                profile[perm] = np.random.choice([0, 1], p=[0.1, 0.9])
            else:
                profile[perm] = np.random.choice([0, 1], p=[0.4, 0.6])
        else:
            # Benign apps: minimal dangerous perms, standard INTERNET usage
            if perm == 'android.permission.INTERNET':
                profile[perm] = np.random.choice([0, 1], p=[0.1, 0.9])
            elif perm in ['android.permission.SYSTEM_ALERT_WINDOW', 'android.permission.REQUEST_INSTALL_PACKAGES']:
                profile[perm] = np.random.choice([0, 1], p=[0.95, 0.05])
            else:
                profile[perm] = np.random.choice([0, 1], p=[0.8, 0.2])

    # --- API Calls Generation ---
    for api in API_CALLS:
        if is_malicious:
            # Higher use of SMS/Telephony/Reflection APIs
            base_risk = random.uniform(0.4, 1.0)
        else:
            base_risk = random.uniform(0.0, 0.3)
        profile[api] = round(base_risk, 4)

    # --- Metadata Generation ---
    if is_malicious:
        profile['app_size_mb'] = round(random.uniform(1.0, 15.0), 2) # Malware often smaller/lighter
        profile['min_sdk'] = random.randint(19, 23) # Targeting older phones
        profile['target_sdk'] = random.randint(28, 30)
        profile['cert_age_days'] = random.randint(0, 30) # Brand new certs = suspicious
        profile['is_debuggable'] = np.random.choice([0, 1], p=[0.7, 0.3])
        profile['obfuscation_score'] = round(random.uniform(0.7, 1.0), 2) # Highly obfuscated
    else:
        profile['app_size_mb'] = round(random.uniform(10.0, 150.0), 2)
        profile['min_sdk'] = random.randint(21, 26)
        profile['target_sdk'] = random.randint(31, 34) # Modern standards
        profile['cert_age_days'] = random.randint(365, 2000) # Established dev
        profile['is_debuggable'] = 0
        profile['obfuscation_score'] = round(random.uniform(0.0, 0.4), 2)

    # --- Behavioral Generation ---
    for beh in BEHAVIOR:
        if is_malicious:
            profile[beh] = round(random.uniform(0.6, 1.0), 4)
        else:
            profile[beh] = round(random.uniform(0.0, 0.3), 4)

    # --- Rule Injection (Enforcing Logic) ---
    # Example: If it asks for SMS permission AND has high overlay score -> Force Malicious
    if profile['android.permission.READ_SMS'] == 1 and profile['screen_overlay_score'] > 0.8:
        profile['label'] = 1

    return profile

# --- Main Execution ---
data = []
for _ in range(int(NUM_SAMPLES * BENIGN_RATIO)):
    data.append(generate_app_profile(is_malicious=False))

for _ in range(int(NUM_SAMPLES * (1 - BENIGN_RATIO))):
    data.append(generate_app_profile(is_malicious=True))

df = pd.DataFrame(data)

# Shuffle dataset
df = df.sample(frac=1).reset_index(drop=True)

# Save
df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Successfully generated {len(df)} samples in {OUTPUT_FILE}")
print(f"Distribution:\n{df['label'].value_counts()}")

✅ Successfully generated 10000 samples in fake_banking_app_dataset.csv
Distribution:
label
1    5000
0    5000
Name: count, dtype: int64


In [4]:
def generate_app_profile(is_malicious):
    profile = {}

    # --- DECIDE SUB-TYPE (To create variance) ---
    # 1. Standard: The typical profile (Easy to detect)
    # 2. Stealth/Sloppy: The "Hard" examples (Confusing for the model)
    subtype = np.random.choice(['standard', 'hard_case'], p=[0.7, 0.3])

    profile['label'] = 1 if is_malicious else 0

    # --- Permissions Generation (More Overlap) ---
    for perm in PERMISSIONS:
        if is_malicious:
            if subtype == 'hard_case':
                # Smart Malware: Requests FEWER permissions to avoid detection
                profile[perm] = np.random.choice([0, 1], p=[0.7, 0.3])
            else:
                # Dumb Malware: Asks for everything
                if perm in ['android.permission.READ_SMS', 'android.permission.SYSTEM_ALERT_WINDOW']:
                    profile[perm] = np.random.choice([0, 1], p=[0.1, 0.9])
                else:
                    profile[perm] = np.random.choice([0, 1], p=[0.4, 0.6])
        else:
            if subtype == 'hard_case':
                # Sloppy Benign: Poorly coded app asks for too much
                profile[perm] = np.random.choice([0, 1], p=[0.8, 0.2])
            else:
                # Good Benign: Minimal permissions
                if perm == 'android.permission.INTERNET':
                    profile[perm] = 1
                else:
                    profile[perm] = np.random.choice([0, 1], p=[0.95, 0.05])

    # --- Metadata Generation (Blurring the Lines) ---
    if is_malicious:
        if subtype == 'hard_case':
            # Sophisticated Malware: Bloated size, older fake cert
            profile['app_size_mb'] = round(random.uniform(40.0, 90.0), 2) # Looks real
            profile['cert_age_days'] = random.randint(100, 400) # "Aged" account
            profile['obfuscation_score'] = round(random.uniform(0.1, 0.5), 2) # Less obvious
        else:
            # Standard Malware
            profile['app_size_mb'] = round(random.uniform(1.0, 15.0), 2)
            profile['cert_age_days'] = random.randint(0, 30)
            profile['obfuscation_score'] = round(random.uniform(0.7, 1.0), 2)
    else:
        if subtype == 'hard_case':
            # New Startup App: Small size, brand new cert (but benign)
            profile['app_size_mb'] = round(random.uniform(2.0, 20.0), 2)
            profile['cert_age_days'] = random.randint(1, 60) # Brand new
            profile['obfuscation_score'] = round(random.uniform(0.5, 0.8), 2) # DRM protection
        else:
            # Standard Benign
            profile['app_size_mb'] = round(random.uniform(30.0, 150.0), 2)
            profile['cert_age_days'] = random.randint(365, 2000)
            profile['obfuscation_score'] = round(random.uniform(0.0, 0.3), 2)

    # --- API Calls (Adding Noise) ---
    for api in API_CALLS:
        noise = np.random.normal(0, 0.05) # Add random jitter
        if is_malicious:
            base = random.uniform(0.4, 1.0) if subtype == 'standard' else random.uniform(0.2, 0.6)
        else:
            base = random.uniform(0.0, 0.3) if subtype == 'standard' else random.uniform(0.1, 0.5)

        profile[api] = np.clip(base + noise, 0, 1)

    # --- Behavioral (Critical Overlap) ---
    # Some benign apps (like legitimate banking apps) DO use overlays or read SMS for OTP
    if is_malicious:
        profile['sms_trigger_score'] = random.uniform(0.6, 1.0)
    else:
        # 20% of benign apps are "OTP Readers" (Legit feature)
        if random.random() < 0.2:
            profile['sms_trigger_score'] = random.uniform(0.5, 0.8)
        else:
            profile['sms_trigger_score'] = random.uniform(0.0, 0.2)

    return profile

In [10]:
!pip install skl2onnx
!pip install onnxruntime

import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time

# --- CONFIGURATION ---
DATASET_FILE = "fake_banking_app_dataset.csv"
MODEL_FILENAME = "fake_app_detector.json" # Native XGBoost save
ONNX_FILENAME = "fake_app_detector.onnx"
TEST_SIZE = 0.2
RANDOM_STATE = 42

print("="*60)
print("🚀 STARTING PROFESSIONAL TRAINING PIPELINE - TEAM ROCKET")
print("="*60)

# 1. LOAD DATA
print(f"\n[1/6] Loading Dataset from {DATASET_FILE}...")
try:
    df = pd.read_csv(DATASET_FILE)
    print(f"   ✅ Loaded {len(df)} samples.")
except FileNotFoundError:
    print("   ❌ Error: Dataset not found! Run the generation script first.")
    exit()

# Separating Features and Target
X = df.drop(columns=['label'])
y = df['label']

# Split into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
print(f"   ✅ Data Split: {len(X_train)} Training samples, {len(X_test)} Test samples.")

# 2. TRAIN XGBOOST MODEL (Option B)
print("\n[2/6] Training XGBoost Classifier...")
start_time = time.time()

# Professional Hyperparameters (Tuned for stability on synthetic data)
model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=RANDOM_STATE
)

model.fit(X_train, y_train)
train_time = time.time() - start_time
print(f"   Model Trained in {train_time:.2f} seconds.")

# 3. EVALUATION
print("\n[3/6] Evaluating Model Performance...")
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_prob)

print(f"   🔹 Accuracy: {acc*100:.2f}%")
print(f"   🔹 ROC-AUC Score: {roc:.4f}")
print("\n   🔹 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Benign (0)', 'Malicious (1)']))

# Confusion Matrix Visualization (Optional: Saves image for report)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Benign', 'Fake'], yticklabels=['Benign', 'Fake'])
plt.title('Confusion Matrix: Fake Banking App Detection')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.savefig('confusion_matrix.png')
print("    Confusion Matrix saved as 'confusion_matrix.png'")

# Feature Importance (Crucial for Explainability)
importance = model.feature_importances_
feature_names = X.columns
feat_importances = pd.Series(importance, index=feature_names)
print("\n   🔹 Top 5 Most Suspicious Features:")
print(feat_importances.nlargest(5))

# 4. EXPORT TO ONNX (The Mandatory Requirement)
print(f"\n[4/6] Exporting to ONNX format ({ONNX_FILENAME})...")

# Define input type (Crucial step: dimensions must match feature count)
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert
onnx_model = convert_sklearn(model, initial_types=initial_type, target_opset=13) # Specify target_opset

# Save file
with open(ONNX_FILENAME, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"    ONNX Model successfully saved to: {ONNX_FILENAME}")

# 5. VERIFY ONNX MODEL (Inference Check)
print("\n[5/6] Verifying ONNX Inference integrity...")

# Start ONNX Runtime session
sess = rt.InferenceSession(ONNX_FILENAME)
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# Run inference on 5 test samples
X_test_sample = X_test.head(5).astype(np.float32).values
pred_onnx = sess.run([label_name], {input_name: X_test_sample})[0]

print("   🔹 Prediction Comparison (XGBoost vs ONNX):")
print(f"      Original: {y_pred[:5]}")
print(f"      ONNX:     {pred_onnx}")

if np.array_equal(y_pred[:5], pred_onnx):
    print("    SUCCESS: ONNX predictions match original model exactly.")
else:
    print("    WARNING: Minor discrepancy in predictions (check float precision).")

print("\n" + "="*60)
print(" PROCESS COMPLETE. READY FOR SUBMISSION.")
print("="*60)

ModuleNotFoundError: No module named 'skl2onnx.common.constants'